In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) # makes the notebook fill the whole window

import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from matplotlib import ticker
from matplotlib.collections import PatchCollection
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import Polygon
from sklearn.cluster import DBSCAN
import datetime
import pandas as pd
import time
import shapefile
import os
from os import listdir
from os.path import isfile, isdir, join
import CubicSplineTrack
#(c)2015 Robert Sedgewick, Kevin Wayne, and Robert Dondero
from greatcircle import great_circle
from datetime import date
import shutil
import imageio

from sklearn.cluster import DBSCAN

import warnings
warnings.filterwarnings("ignore")

#Turn off interactive plotting for pyplot
plt.ioff()

In [2]:
"""Team WAAAAAAEEEEE Cassie"""
check_minutes = 60 # minuite var
split_storm = 1 # seperate or no

only_dirs = [d for d in listdir("Data/") if isdir(join("Data/", d))] # makes a list of all the folders in Data/
dir_count = len(only_dirs)

WWLLN_Track_Path = [""] * dir_count # the paths of all the track files, in the same order as only_dirs
WWLLN_Locations_Path = [""] * dir_count # the paths of all the lightning locations files, in the same order as only_dirs
Cubic_Spline_Path = [""] * dir_count # the paths of all the cubic spline files, in the same order as only_dirs

Image_Path = [""] * dir_count # Base Image paths
Storm_Names = [""] * dir_count # Base storm names
T_Path = [""] * dir_count

# loop thru all dirs
storms_found = 0
dir_indx = 0
while dir_indx < dir_count:
    
    # this gets all the file paths in a given storms' folder
    only_dirs[dir_indx] = join("Data/", only_dirs[dir_indx])
    file_paths = listdir(only_dirs[dir_indx])
    
    # check if the 3 nessisary files exist
    if len(file_paths) < 2:
        only_dirs[dir_indx] = ""
        dir_indx+=1
        continue
    
    #print(only_dirs[dir_indx])
    #print(file_paths)
    
    # for every file in file_paths, sort it into its' respective arrays
    file_indx = 0
    files_found = 0
    Cubic_Found = False
    while file_indx < len(file_paths):
        # creates the files full path
        file_paths[file_indx] = join(only_dirs[dir_indx]+"/", file_paths[file_indx])
        #print(file_paths[file_indx])
        
        # sorts the file path based on the last expected word of a file and it's type
        if file_paths[file_indx].endswith("Trackfile.txt"):
            WWLLN_Track_Path[dir_indx] = file_paths[file_indx]
            files_found+=1
            
            # generates the name of the storm for images
            Storm_Names[dir_indx] = "Hurricane " + WWLLN_Track_Path[dir_indx].rsplit("_", 3)[1]
            
        elif file_paths[file_indx].endswith("Locations.txt"):
            WWLLN_Locations_Path[dir_indx] = file_paths[file_indx]
            files_found+=1
            
        elif file_paths[file_indx].endswith("Trackfile.csv"):
            Cubic_Spline_Path[dir_indx] = file_paths[file_indx]
            Cubic_Found = True
            files_found+=1
        #print(f"File: {file_paths[file_indx]}, Storm indx: {dir_indx}\n")
        file_indx+=1
    
    # Run with Bens' Cubic Spline
    if not Cubic_Found:
        Cubic_Spline_Path[dir_indx] = join(only_dirs[dir_indx]+"/", Storm_Names[dir_indx] + " Cubic Spline Trackfile.csv")
        #print(WWLLN_Track_Path[dir_indx])
        CubicSplineTrack.cubic_spline_trackfile(WWLLN_Track_Path[dir_indx], Cubic_Spline_Path[dir_indx], check_minutes)
        files_found+=1
    
    if files_found != 3:
        print(f"Incorrect files found: {files_found}, {file_paths}\n")
        only_dirs[dir_indx] = ""
        dir_indx+=1
        continue
    
    # makes the path for this index's storm
    Image_Path[dir_indx] = join("./Images/", Storm_Names[dir_indx])
    
    if not os.path.exists(Image_Path[dir_indx]):
        os.mkdir(Image_Path[dir_indx])
    
    T_Path[dir_indx] = Image_Path[dir_indx] + "/T Value"
    Image_Path[dir_indx] = Image_Path[dir_indx] + "/" + Storm_Names[dir_indx]
    
    if split_storm:
        Image_Path[dir_indx] = Image_Path[dir_indx] + " Split " + str(check_minutes) +" Minutes"
    else:
        Image_Path[dir_indx] = Image_Path[dir_indx] + " " + str(check_minutes) +" Minutes"
    
    # if an image folder |dosen't| exist, create a new one with the path at Image_Path[dir_indx]
    if not os.path.exists(Image_Path[dir_indx]):
        os.mkdir(Image_Path[dir_indx])
    
    Image_Path[dir_indx] = Image_Path[dir_indx] + '/' # add a slash for use in the next block, making the actual img paths
    
    if not os.path.exists(T_Path[dir_indx]):
        os.mkdir(T_Path[dir_indx])
        T_Path[dir_indx] = T_Path[dir_indx] + '/'
        if not os.path.exists(T_Path[dir_indx]+"0.0"):
            os.mkdir(T_Path[dir_indx]+"0.0")
        T_value = 1.5
        while T_value < 9:
            if not os.path.exists(T_Path[dir_indx]+str(T_value)):
                os.mkdir(T_Path[dir_indx]+str(T_value))
            T_value+=0.5
    else:
        T_Path[dir_indx] = T_Path[dir_indx] + '/'
    
    #print('\n')
    storms_found+=1
    dir_indx+=1

if not os.path.exists("Images/T Value"):
    os.mkdir("Images/T Value")
    if not os.path.exists("Images/T Value/0.0"):
        os.mkdir("Images/T Value/0.0")
    T_value = 1.5
    while T_value < 9:
        if not os.path.exists("Images/T Value/"+str(T_value)):
            os.mkdir("Images/T Value/"+str(T_value))
        T_value+=0.5

# print for debug
print(f"Storms Found: {storms_found}")
print(Storm_Names)
#print(Image_Path)
#print(WWLLN_Track_Path)
#print(WWLLN_Locations_Path)
#print(Cubic_Spline_Path)

Storms Found: 34
['Hurricane Arthur', 'Hurricane Bertha', 'Hurricane Beta', 'Hurricane Cristobal', 'Hurricane Delta', 'Hurricane Dolly', 'Hurricane Edouard', 'Hurricane Epsilon', 'Hurricane Eta', 'Hurricane Fay', 'Hurricane Five', 'Hurricane Gamma', 'Hurricane Gonzalo', 'Hurricane Hanna', 'Hurricane Iota', 'Hurricane Isaias', 'Hurricane Josephine', 'Hurricane Kyle', 'Hurricane Laura', 'Hurricane Marco', 'Hurricane Nana', 'Hurricane Omar', 'Hurricane Paulette', 'Hurricane Rene', 'Hurricane Sally', 'Hurricane Teddy', 'Hurricane Ten', 'Hurricane Theta', 'Hurricane Twenty', 'Hurricane Twenty-Eig', 'Hurricane Twenty-Nin', 'Hurricane Vicky', 'Hurricane Wilfred', 'Hurricane Zeta']


In [6]:
dir_indx = 0
while dir_indx < dir_count:
    with imageio.get_writer(Image_Path[dir_indx]+'All.gif', mode='I') as writer:
        Images = [f for f in listdir(Image_Path[dir_indx]) if isfile(join(Image_Path[dir_indx], f)) and f.endswith(".png")]
        img_cnt = len(Images)
        img_indx = 0
        while img_indx < img_cnt:
            image = imageio.imread(Image_Path[dir_indx] + str(img_indx) + ".png")
            writer.append_data(image)
            img_indx+=1
    dir_indx+=1

In [3]:
"""Team WAAAAAEEEEEEEEE Lance"""
def run_circle(row, center_lat, center_lon): 
    lat = (row['Lat'])
    lon = (row['Long'])
    return great_circle(lat, lon, center_lat, center_lon)

In [4]:
"""Team WAAAAAAEEEEEEEEEE Bhargav"""
#Creating intensity table
intensity_values = {
     'T-Number': [1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5],
  '1-min Winds': [25, 30, 35, 45, 55, 65, 77, 90, 102, 115, 127, 140, 155, 170, 185],
'Min. Pressure': [0, 1009, 1005, 1000, 994, 987, 979, 970, 960, 948, 935, 921, 906, 890, 873]
}
T_frame = pd.DataFrame(intensity_values)

def Tvalue(wind_speed):
    i=0
    half=0
    current = 0
    prev=0
    after=0
    for i in range(15):
        current = T_frame.loc[i, '1-min Winds']
        if (i == 0):
            prev=20
        else:
            prev = T_frame.loc[(i-1), '1-min Winds']
        
        if(i==14):
            after = 188
            
        else:
            after=T_frame.loc[(i+1), '1-min Winds']

        #if(wind_speed < 25):
            #return 1.5
        
        if(wind_speed > 185):
            return 8.5
        
        else:
            half_prev =current - ((current-prev)/2)
            half_aft = current + ((after-prev)/2)
            if(wind_speed >= half_prev and wind_speed < half_aft):
                return T_frame.loc[i, 'T-Number']
    return 0

In [7]:
"""
TODO:
-cite sources (great circle etc.)
-sort output by intensity (Dvorac scale?) 1-5
-statitics over a single storm and multiple storms
-faster file save with PIL?
"""
# for every storm, do the things
"""Team WAAAAAAAEEEEEE Cassie"""
dir_indx = 0
while dir_indx < dir_count:
    # if storm is invalid, skip doin the things
    if only_dirs[dir_indx] == "":
        dir_indx+=1
        continue
    
    """Team Squeem"""
    # This loads the WWLLN trackfile Jeremy and Natalia give us
    # wind = nautical mph, pressure = millibars
    df = pd.read_csv(WWLLN_Track_Path[dir_indx], header=None, names=["Year","Month","Day","Hour","Lat","Long","Min_Pressure","Max_Winds","Unused"], low_memory=False, sep='\t')
    df = df.drop("Unused", axis=1)#data filtering
    df['Date'] = df.apply(lambda x: pd.to_datetime(f"{int(x['Year'])}-{int(x['Month'])}-{int(x['Day'])}-{int(x['Hour'])}", format="%Y-%m-%d-%H"), axis=1)
    
    "Team WAAAAAEEEEE Lance"
    #wind pressure stuff 
    #run pressure wind speed function on df for every row, save into array
    Tindex = 0
    Tvalues = df['Max_Winds'].apply(Tvalue)
    #Tvalues = [Tvalue(t) for t in df['Max_Winds'] if isdir(join("Data/", d))]
    #make dataframe 
    dates = df['Date']
    #print(Tvalues)
    #print(df['Max_Winds'])
    if len(Tvalues) != len(dates):
        raise Exception("Lengths differ!")
    #loop through df, for every row, check date 
    df_length = len(dates)
    left_half_dates = [date.min] * df_length
    right_half_dates = [date.min] * df_length
    date_index = 1 
    left_half_dates[0] = date.min
    right_half_dates[df_length - 1] = date.max
    while date_index < df_length: 
        left_half_dates[date_index] = dates.iloc[date_index - 1] + ((dates.iloc[date_index] - dates.iloc[date_index - 1]) / 2)
        date_index += 1
    date_index = 0
    while date_index < df_length - 1: 
        right_half_dates[date_index] = dates.iloc[date_index] + ((dates.iloc[date_index] - dates.iloc[date_index + 1]) / 2)
        date_index += 1
    
    """Team Squeem"""
    # This loads the WWLLN lighting location data that Jeremy and Natalia give us
    ln = pd.read_csv(WWLLN_Locations_Path[dir_indx], header=None, names=["Year","Month","Day","Hour","Min","Sec","Lat","Long","Dist_East_West","Dist_North_South"], low_memory=False, sep=' ')
    ln['Date'] = ln.apply(lambda x: pd.to_datetime(f"{int(x['Year'])}-{int(x['Month'])}-{int(x['Day'])}-{int(x['Hour'])}-{int(x['Min'])}-{x['Sec']}", format="%Y-%m-%d-%H-%M-%S.%f"), axis=1)#is a data clean

    # This opens the cubic spline trackfiles that Ben creates
    storm_center = pd.read_csv(Cubic_Spline_Path[dir_indx])
    storm_center['Date'] = storm_center['Date'].apply(pd.to_datetime)

    # Genereates the list of dates from the WWLLN trackfile of a (check_minutes) range
    dates = pd.date_range(df['Date'][0], df['Date'][len(df)-1], freq=str(check_minutes)+"T")
    
    buffer = 10
    #if split_storm:
        #buffer = 6 # affects map zooooooooooooom
    shp_file = "World_Countries__Generalized_-shp/World_Countries__Generalized_" # Map file ???????
    cmap = 'gist_ncar' # dunno WHAT this is

    # Counter and iteration
    for index, date in enumerate(dates): # this loop makes one image for every (check_minutes) amount of time
        starttime = time.perf_counter() # speeed timer start
        center = storm_center[storm_center['Date'] == date] # storm center at this time frame
        """Team WAAAAAAEEEEE Cassie and Lance"""
        if split_storm:
            ln_curr_timeframe = ln[(ln['Date'] >= date) & (ln['Date'] < date + datetime.timedelta(minutes=check_minutes))]
            #print(ln_curr_timeframe)
            if len(ln_curr_timeframe) != 0:
                ln_curr_timeframe['Distance'] = ln_curr_timeframe.apply(run_circle, axis = 1, args = ((center['Lat']), (center['Long'])))
                #print(ln_curr_timeframe)
                subset = ln_curr_timeframe[(ln_curr_timeframe['Distance'] <= 100 / 1.852) | ((ln_curr_timeframe['Distance'] >= 200 / 1.852) & (ln_curr_timeframe['Distance'] <= 400 / 1.852))]
            else:
                subset = ln_curr_timeframe
        else:
            # Select just the lightning locations that are within (check_minutes) minutes
            subset = ln[(ln['Date'] >= date) & (ln['Date'] < date + datetime.timedelta(minutes=check_minutes))]
        
        """Team WAAAAAAEEEEEE Cassie"""
        if date >= right_half_dates[Tindex]:
            Tindex += 1
        stormTval = Tvalues[Tindex]
        
        # if lightning exists, do dbscan on it
        if len(subset != 0):
            dbscan = DBSCAN(eps=.2, min_samples=3)
            subset['Labels'] = dbscan.fit_predict(subset[['Long','Lat']])
        
        """Team Squeem"""
        # Setup the figure
        fig, ax = plt.subplots(figsize=(15,15))
        
        # this is for the coastline
        # This reads the shapefile, extracts each shape, creates a polygon, and adds it to the list
        sf = shapefile.Reader(shp_file)
        shapes = sf.shapes()
        Nshp = len(shapes)
        ptchs = []
        for nshp in range(Nshp):
            pts = np.array(shapes[nshp].points)
            prt = shapes[nshp].parts
            par = list(prt) + [pts.shape[0]]
            for pij in range(len(prt)):# <--------------------------------------------------------------------------------------------------------------------------- FIX ME (im slow) use while loops, never range()!
                ptchs.append(Polygon(pts[par[pij]:par[pij+1]]))
        # Every every polygon from the shapefile to figure
        ax.add_collection(PatchCollection(ptchs, facecolor= '#838688', edgecolor='k', linewidths=1., zorder=2))

        # This adds the water color
        ax.add_patch(mpl.patches.Rectangle((-180,-89),360,180,color='#a6cae0'))

        # This connects Antarcitca to the bottom of the image
        ax.add_patch(mpl.patches.Rectangle((-179.9,-89.9),360,2,color='#838688',zorder=3))

        # These are more general image settings
        plt.title(Storm_Names[dir_indx]+" T"+str(stormTval)+' '+str(date), fontdict = {'fontsize' : 40})

        #edges = (-180,180,-90,90) # Left, Right, Bottom, Top | Set this if you want to change the map scale
        edges = (center['Long'].values[0] - buffer, center['Long'].values[0] + buffer, center['Lat'].values[0] - buffer, center['Lat'].values[0] + buffer) # <--------------------------This affects map zoooooooooom

        # setting up img stuff, prob shouldn't be touched
        xlim = np.append(np.arange(edges[0], edges[1], step=2.5), edges[1])
        ylim = np.append(np.arange(edges[2], edges[3], step=2.5), edges[3])
        ax.set_xticks(xlim)
        ax.set_yticks(ylim)
        ax.set_xlim(edges[0], edges[1])
        ax.set_ylim(edges[2], edges[3])
        ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
        ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
        plt.rc('xtick', labelsize=30)
        plt.rc('ytick', labelsize=30)
        
        """Team WAAAAAEEEEEE Cassie"""
        # if lightning exists, put it in the image
        if len(subset) != 0:
            plt.scatter(subset['Long'], subset['Lat'], c=subset['Labels'], cmap=cmap, zorder=4) # plots the dbscan data
        
        """Team Squeem"""
        #plt.scatter(subset['Long'],subset['Lat'],s=100,c="yellow",edgecolors='black',zorder=5) # <- old plot func for non-grouped data
        plt.scatter(center['Long'], center['Lat'], s=140, c="red", edgecolors='black' ,zorder=5, alpha=0.5) # plots the center of the storm

        # save and close the image
        plt.savefig(Image_Path[dir_indx] + str(index), bbox_inches='tight', pad_inches=.4)
        plt.savefig(T_Path[dir_indx] + str(stormTval) + '/' + Storm_Names[dir_indx] + ' ' + str(index), bbox_inches='tight', pad_inches=.4)
        plt.savefig("Images/T Value/" + str(stormTval) + '/' + Storm_Names[dir_indx] + ' ' + str(index), bbox_inches='tight', pad_inches=.4)
        plt.close('all')

        # Shows roughly how long we have left
        taken = time.perf_counter() - starttime
        print(f"Storm #{dir_indx}: {Storm_Names[dir_indx]}, T{stormTval}\tPercent Done: {(index + 1) / len(dates) * 100 : .2f}%\tTime taken: {taken : .2f} seconds\tEst time remaining: {datetime.timedelta(seconds=(len(dates) - (index + 1)) * taken)} {len(subset)}", end='\r')
    dir_indx+=1

,T-Number,1-min Winds,Min. Pressure
0,1.5,25,0
1,2.0,30,1009
2,2.5,35,1005
3,3.0,45,1000
4,3.5,55,994
5,4.0,65,987
6,4.5,77,979
7,5.0,90,970
8,5.5,102,960
9,6.0,115,948
